In [1]:
import sys
from collections import OrderedDict
import matplotlib.pyplot as plt
import parselmouth
import re
import pandas as pd

import librosa
import numpy as np

In [2]:
def run_script_with_pluseq(audio_file_path):
    # Generate data and profile_data files using prosogram in praat    
    init_script = 'include prosomain.praat\n'
    init_script += 'call prosogram file=' + file_in + ' save=yes draw=no\n'
    init_script += 'exit'
    script = init_script
    to_include = True
    while to_include:
        match = re.search(r'^include ((?:\w|\.)+)', script, re.MULTILINE)
        if match:
            with open(match.group(1)) as f:
                script = script[:match.start()] + f.read() + '\n' + script[match.end():]
        else:
            to_include = False

    script = re.sub(r'((?:\w|\.)+\$)\s*\+=', r'\1 = \1 +', script)
    parselmouth.praat.run(script)

In [3]:
%%capture
name = "telugu_para2"
wav_file =name+".wav"
file_in = './wav_files/'+wav_file
run_script_with_pluseq(file_in)

Batch mode...
 
Batch command prosogram
Processing input file <telugu_para2>...
Selected optimal segmentation method: asyll
Analysis time range: 0 - 66.471 s (1 min, 6.471 s)
Reading parameter files...
Consulted pitch from file <./wav_files/telugu_para2.Pitch> (time step=0.005)
Consulted BP intensity from file <./wav_files/telugu_para2_BP.Intensity> (time step=0.005)
Calculating parameters...
Segmentation into syllabic nuclei. Method=asyll, Time range=0-66.471
Calculating actual segmentation. Method=asyll...
Detecting hesitations (method=automatic)...
Calculating stylization... (asyll, G(adapt)=0.16-0.32/T^2, DG=30, dmin=0.035)
Calculating stylization... Ready
No speaker tier in input textgrid. Assuming 1 speaker.
Calculating pitch range of each speaker...
Calculating pitch range normalized pitch...
Writing prosodic profile report for current input file to: ./wav_files/telugu_para2_profile.txt
Writing prosodic profile of current input file to: ./wav_files/telugu_para2_profile_data.txt


In [4]:
#Now using the datafiles for patterns

def process_datafiles(file_in):
    datafile_in = file_in[:-4] + "_data.txt" #removing ".wav"

    f = open(datafile_in)

    text = f.read()
    lines = text.split('\n')

    new_lines = []

    for line in lines:
        split_line = line.split('\t')
        new_lines.append(split_line)

    row_label_index = new_lines[0].index("rowLabel")
    nucl_t1_index = new_lines[0].index("nucl_t1")
    nucl_t2_index = new_lines[0].index("nucl_t2")
    f0_start_index = new_lines[0].index("f0_start")
    f0_end_index = new_lines[0].index("f0_end")
    f.close()
    return new_lines,row_label_index,f0_start_index,f0_end_index

In [19]:
def process_profile_data(file_in):
    profile_in = file_in[:-4] + "_profile_data.txt"

    fp = open(profile_in)

    text = fp.read()

    lines = text.split('\n')

    new_profile_lines = []

    for line in lines:
        split_line = line.split('\t')
        new_profile_lines.append(split_line)

    speech_rate = new_profile_lines[1][new_profile_lines[0].index("SpeechRate")]
    speech_time = new_profile_lines[1][new_profile_lines[0].index("SpeechTime")]
    f0_mean = float(new_profile_lines[1][new_profile_lines[0].index("F0MeanHz")])
    pitch_top = float(new_profile_lines[1][new_profile_lines[0].index("PitchTopHz")])
    pitch_bottom = float(new_profile_lines[1][new_profile_lines[0].index("PitchBottomHz")])

    fp.close()
    
    return f0_mean,pitch_top,pitch_bottom

In [22]:
new_lines,row_label_index,f0_start_index,f0_end_index = process_datafiles(file_in)
# print(row_label_index,f0_start_index,f0_end_index)
# print(new_lines[:2])
f0_mean,pitch_top,pitch_bottom = process_profile_data(file_in)
# print(f0_mean,pitch_top,pitch_bottom )

0 8 9
[['rowLabel', 'nucl_t1', 'nucl_t2', 'f0_min', 'f0_max', 'f0_median', 'f0_mean', 'f0_meanST', 'f0_start', 'f0_end', 'lopitch', 'hipitch', 'hipitchST', 'dynamic', 'intrasyllab', 'intersyllab', 'up', 'down', 'trajectory', 'f0_discont', 'prnp_start', 'prnp_end', 'prnp_intra', 'nucl_dur', 'syll_dur', 'vowel_dur', 'rhyme_dur', 'gap_left', 'loudness', 'int_peak', 'hesitation', 'speaker_id', 'before_pause', 'after_pause', 'pause_dur', 'iso_dur'], ['0.535', '0.5354081632653017', '0.6104081632653018', '96', '115', '109', '108', '81.19', '109', '109', '109', '109', '81.28', '0', '0', '0', '0', '0', '0', '0', '44', '44', '0', '0.07500000000000007', '0', '0', '0', '0.5354', '0', '80', '0', '1', '0', '1', '0', '0.16000000000000003']]


In [8]:
def comparison(f0_value, pitch_top, pitch_bottom, f0_mean):
    L_value = abs(f0_value - pitch_bottom)
    M_value = abs(f0_value - f0_mean)
    H_value = abs(f0_value - pitch_top)

    if L_value <= M_value and L_value < H_value:
        return "L"
    elif M_value < L_value and M_value <= H_value:
        return "M"
    else:
        return "H"

In [23]:
#Getting L,H,M from data

def get_labelled_rows(new_lines,row_label_index,f0_start_index,f0_end_index,f0_mean,pitch_top,pitch_bottom):
    new_lines = new_lines[1:] #remove header

    labelled_rows = OrderedDict()

    for row in new_lines:
        if len(row) > 1:
            #print("row_label:", row[row_label_index])
            #print("f0_start:", row[f0_start_index])
            #print("f0_end:", row[f0_end_index])

            label = row[row_label_index]
            f0_start = float(row[f0_start_index])
            f0_end = float(row[f0_end_index])
            #nucl_t1 = float(row[nucl_t1_index])
            #nucl_t2 = float(row[nucl_t2_index])


            if f0_start == f0_end:
                result_pitch = comparison(f0_start, pitch_top, pitch_bottom, f0_mean)
                #print(result_pitch)
                labelled_rows[label] =  result_pitch#, nucl_t1, nucl_t2]

            else:
                start_label = comparison(f0_start, pitch_top, pitch_bottom, f0_mean)
                end_label = comparison(f0_end, pitch_top, pitch_bottom, f0_mean)
                result_pitch = start_label + "-" + end_label
                #print(result_pitch)
                labelled_rows[label] =  result_pitch#, nucl_t1, nucl_t2]
    return labelled_rows
#     print(labelled_rows)

In [24]:
labelled_rows = get_labelled_rows(new_lines,row_label_index,f0_start_index,f0_end_index,f0_mean,pitch_top,pitch_bottom)

In [25]:
def visualising_data(labelled_rows):
    #Visualising the Data
    y_array = []

    for i in labelled_rows.keys():
        if labelled_rows[i] == 'H' or labelled_rows[i] == 'H-H':
            y_array.append(float(1))
        elif labelled_rows[i] == 'M' or labelled_rows[i] == 'M-M':
            y_array.append(float(0))
        elif labelled_rows[i] == 'L' or labelled_rows[i] == 'L-L':
            y_array.append(float(-1))
        elif labelled_rows[i] == 'L-M' or labelled_rows[i] == 'M-L':
            y_array.append(float(-0.5))
        elif labelled_rows[i] == 'M-H' or labelled_rows[i] == 'H-M':
            y_array.append(float(0.5))
        elif labelled_rows[i] == 'L-H' or labelled_rows[i] == 'H-L':
            y_array.append(float(0))
        else:
            y_array.append(float(0))
    return y_array

In [34]:
y_array = visualising_data(labelled_rows)
list_x =(list(labelled_rows.keys()))
x = [float(j) for j in list_x]
data = {'Time Stamps (s)': x, 'Values': y_array, 'Labels': [labelled_rows["{:.3f}".format(row)] for row in x]}
df = pd.DataFrame(data)
df["punc"] = None
df

,Time Stamps (s),Values,Labels,punc
0,0.535,0.0,M,None
1,0.695,0.0,M,None
2,0.895,-1.0,L,None
3,1.025,-1.0,L,None
4,1.255,-1.0,L,None
...,...,...,...,...
294,65.225,-1.0,L,None
295,65.385,-0.5,M-L,None
296,65.620,-0.5,M-L,None
297,65.760,-1.0,L,None


In [ ]:
for num in range(2,10):
    

In [35]:
# Function to find the index of the closest timestamp in list1 for a given timestamp
def find_closest_index(target, lst):
    return np.abs(np.array(lst) - target).argmin()

In [ ]:
concatenated_df = pd.concat(df_list, ignore_index=True)
list1 = df['Time Stamps (s)'].tolist()
list2 = [10.4, 14.4, 18.45, 28.35, 37.9, 41.65, 48.95, 54, 61,66.4]
closest_indices = [find_closest_index(timestamp, list1) for timestamp in list2]

In [38]:
l2 = [10.4, 14.4, 18.45, 28.35, 37.9, 41.65, 48.95, 54, 61,66.4]
l3 = [13.2,20.05,28.95,36.4]
l4 = [8.95,19.05,26.3,29.7,34.4]
l5 = [9.05,14.57,19.5,26.54,32.54,37.5,44.67]
l6 = [7.85,16.75,25.13]
l7 = [10.7,20.3,25.6,30.8,36.1,42.76]
l8 = [10.5,17.5,24,34.3]
l9 = [6.5,10.8,15.4,26.7]
stop_list = [l2,l3,l4,l5,l6,l7,l8,l9]

In [44]:
l2=[]
l3=[8.56]
l4=[1.25,2.4,3.55,15.75]
l5=[30.15]
l6=[9.3,10.85,13.1]
l7=[6.5,13.7]
l8=[25.5]
l9=[]
comma_list = [l2,l3,l4,l5,l6,l7,l8,l9]

In [45]:
l2=[2.2]
l3=[1.55]
l4=[]
l5 = [1.57]
l6=[1.4]
l7=[]
l8=[]
l9=[]
ex_list = [l2,l3,l4,l5,l6,l7,l8,l9]

In [46]:
phrase_end = [stop_list,coma_list,ex_list]

In [ ]:
for i in range(3):
    if(i==0):
        punctuation = "."
    elif(i==1):
        punctuation = ","
    elif (i==2):
        punctuation = "!"
    
    list1 = df_list[j]['Time Stamps (s)'].tolist()
    list2 = phrase_end[i][j]
    closest_indices = [find_closest_index(timestamp, list1) for timestamp in list2]
    for i, timestamp in enumerate(list2):
        closest_index = closest_indices[i]
        closest_timestamp = list1[closest_index]
        df_list[j].at[closest_index, "punc"] = punctuation